### 日本語テキストを分かち書きする
- "wakatigaki" of Japanese text  
- Preparation:
  - Setup MeCab + NEologd
    - see ../install_NEologd_on_google_drive.ipynb



#### Set parameters for MeCab

In [3]:
neologd_dir_on_drive = '/content/drive/MyDrive/Documents/ds2024/dsF1/mecab-neologd/'

import os
os.environ["MECABRC"] = '{}/etc/mecabrc'.format(neologd_dir_on_drive)
mecab_dic_path = '-d {}/mecab-ipadic-neologd'.format(neologd_dir_on_drive)

#### Mount google drive

In [4]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Documents/ds2024/dsF1/

Mounted at /content/drive
/content/drive/MyDrive/Documents/ds2024/dsF1


#### Setup for MeCab

In [5]:
!pip install mecab-python3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.7/581.7 kB 7.5 MB/s eta 0:00:00


#### Import libraries

In [6]:
import os
import pandas as pd
import MeCab

# pd.options.display.max_columns=999
# pd.options.display.max_rows=999
# pd.options.display.max_colwidth=999

#### Let's Go!

In [7]:
csv_in = 'sleep-text-score.csv'

In [8]:
df = pd.read_csv(csv_in, sep=',', skiprows=0, header=0, encoding='shift-jis')
print(df.shape)
print(df.info())
display(df.head())
n_texts = df['text'].shape[0]
print('#texts:', n_texts)

(426, 4)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426 entries, 0 to 425
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   text               426 non-null    object
 1   GPT-4o             426 non-null    int64 
 2   Gemini-1.5-Pro     426 non-null    int64 
 3   Claude-3.5-Sonnet  426 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 13.4+ KB
None


,text,GPT-4o,Gemini-1.5-Pro,Claude-3.5-Sonnet
0,就寝時間を毎日一定にする,2,2,2
1,朝日を積極的に浴びる,2,2,2
2,寝室の温度を18-22度に保つ,2,2,2
3,就寝前のストレッチで体をリラックスさせる,2,2,2
4,寝具は定期的に清潔に保つ,2,2,2


#texts: 426


#### 名詞、動詞、形容詞だけに絞って分かち書きする

In [9]:
%%time

results = []
used_features = ['名詞', '動詞', '形容詞']
tagger = MeCab.Tagger(mecab_dic_path)  # MeCabで形態素解析をするときのおまじない
for i in range(n_texts):  # texts中の各文について
  s = df.at[i, 'text']
  node = tagger.parseToNode(s)  # 文中の全単語を順にみるためのおまじない
  wakati = []
  while node:  # 文中の全単語を順にみるためのおまじない
    w = node.surface  # 単語
    if w == '':
      node = node.next
      continue
    features = node.feature.split(',')  # 品詞のリストを作成
    print(w, features)  # debug
    f0 = features[0]
    if f0 in used_features: wakati.append(w)
    node = node.next  # 次の単語へ
  results.append(' '.join(wakati))  # 半角空白を挟んで文字列化

就寝 ['名詞', 'サ変接続', '*', '*', '*', '*', '就寝', 'シュウシン', 'シューシン']
時間 ['名詞', '副詞可能', '*', '*', '*', '*', '時間', 'ジカン', 'ジカン']
を ['助詞', '格助詞', '一般', '*', '*', '*', 'を', 'ヲ', 'ヲ']
毎日 ['名詞', '副詞可能', '*', '*', '*', '*', '毎日', 'マイニチ', 'マイニチ']
一定 ['名詞', 'サ変接続', '*', '*', '*', '*', '一定', 'イッテイ', 'イッテイ']
に ['助詞', '格助詞', '一般', '*', '*', '*', 'に', 'ニ', 'ニ']
する ['動詞', '自立', '*', '*', 'サ変・スル', '基本形', 'する', 'スル', 'スル']
朝日 ['名詞', '固有名詞', '組織', '*', '*', '*', '朝日', 'アサヒ', 'アサヒ']
を ['助詞', '格助詞', '一般', '*', '*', '*', 'を', 'ヲ', 'ヲ']
積極的 ['名詞', '固有名詞', '一般', '*', '*', '*', '積極的', 'セッキョクテキ', 'セッキョクテキ']
に ['助詞', '格助詞', '一般', '*', '*', '*', 'に', 'ニ', 'ニ']
浴びる ['動詞', '自立', '*', '*', '一段', '基本形', '浴びる', 'アビル', 'アビル']
寝室 ['名詞', '一般', '*', '*', '*', '*', '寝室', 'シンシツ', 'シンシツ']
の ['助詞', '連体化', '*', '*', '*', '*', 'の', 'ノ', 'ノ']
温度 ['名詞', '一般', '*', '*', '*', '*', '温度', 'オンド', 'オンド']
を ['助詞', '格助詞', '一般', '*', '*', '*', 'を', 'ヲ', 'ヲ']
18 ['名詞', '数', '*', '*', '*', '*', '*']
-22度 ['名詞', '固有名詞', '一般', '*', '*', '*', '-22度

In [10]:
print(results)

['就寝 時間 毎日 一定 する', '朝日 積極的 浴びる', '寝室 温度 18 -22度 保つ', '就寝 前 ストレッチ 体 リラックス さ せる', '寝具 定期的 清潔 保つ', '夕方 以降 カフェイン 摂取 避ける', '寝室 暗く し 快適 環境 作る', '適度 運動 日中 行う', '就寝 前 深呼吸 リラックス する', '快適 寝具 選ぶ', '白湯 寝る 前 一杯 飲む', '寝室 照明 暖色系 する', '瞑想 取り入れる', '寝る 前 軽い 読書 する', '寝室 観葉植物 置く', 'エッセンシャルオイル 使っ アロマセラピー', 'ドライヤー 音 雨 音 ホワイトノイズ 使う', '寝る 前 ストレッチ', 'カフェイン 摂取 午後2時 終える', '寝る 1時間前 風呂 入る', '寝室 温度 適切 保つ', '寝る 前 スマホ パソコン 避ける', 'ナイトモード ある デバイス 使用 する', '毎朝 時間 起きる', '夜遅く スマートフォン 見る', '不規則 就寝 時間', '寝る 直前 激しい 運動 する', '夜遅く 重い 食事 とる', '昼寝 長時間 とる', '寝室 仕事 する', '就寝 前 エネルギー ドリンク 飲む', '休日 昼過ぎ 寝る', '寝室 温度 管理 し', '就寝 前 ストレス フル 作業 する', '寝る 前 激しい アクション映画 見る', '寝室 明るい 照明 使う', '寝る 前 食事 とる', 'カフェイン 夜遅く 摂取 する', '不規則 寝る時間 続ける', '寝室 仕事 し 続ける', '寝る 前 激しい 運動', '部屋 散らかし まま 睡眠 入る', '寝室 中 明るい スマホ 画面 見 続ける', 'アルコール 摂取 する', 'ネガティブ ニュース 寝る 前 チェック する', '寝る 前 カーボン ハイ 食べ物 食べる', '寝る 前 喧嘩 する', 'ページ めくる の やめ られ スリラー 小説 読む', '昼寝 長時間 する', '寝室 通気 悪く する', '寝室 温度 高く する', '大音量 音楽 寝る 前 聴く', '明日 仕事 こと 心配 する', '人混み 行く', 'SNS 無限 スクロール する', '帰宅 遅く なる', '問題 解決 

In [11]:
df['wakati'] = results
display(df)

,text,GPT-4o,Gemini-1.5-Pro,Claude-3.5-Sonnet,wakati
0,就寝時間を毎日一定にする,2,2,2,就寝 時間 毎日 一定 する
1,朝日を積極的に浴びる,2,2,2,朝日 積極的 浴びる
2,寝室の温度を18-22度に保つ,2,2,2,寝室 温度 18 -22度 保つ
3,就寝前のストレッチで体をリラックスさせる,2,2,2,就寝 前 ストレッチ 体 リラックス さ せる
4,寝具は定期的に清潔に保つ,2,2,2,寝具 定期的 清潔 保つ
...,...,...,...,...,...
421,就寝前に口論する,0,0,0,就寝 前 口論 する
422,深夜にSNSで政治的議論を行う,0,0,0,深夜 SNS 政治的 議論 行う
423,就寝前の激しいストレッチ,0,0,0,就寝 前 激しい ストレッチ
424,夜遅くの重い食事,0,0,0,夜遅く 重い 食事


In [12]:
df = df.drop(columns=['text'])

In [13]:
df = df[['wakati'] + [col for col in df.columns if col != 'wakati']]

In [14]:
df = df.rename(columns={'wakati': 'text'})

In [15]:
display(df.head())

,text,GPT-4o,Gemini-1.5-Pro,Claude-3.5-Sonnet
0,就寝 時間 毎日 一定 する,2,2,2
1,朝日 積極的 浴びる,2,2,2
2,寝室 温度 18 -22度 保つ,2,2,2
3,就寝 前 ストレッチ 体 リラックス さ せる,2,2,2
4,寝具 定期的 清潔 保つ,2,2,2


In [16]:
df.to_csv('sleep-score-text-wakati.csv', index=False)